# Civil War Book Review scratchpad

https://www.reportlab.com/docs/reportlab-userguide.pdf



In [1]:
from collections import namedtuple
import csv
import re
import os

In [2]:
cwbr_dict = dict()

# name the file you wish to examine [Split_Into_Types/---------.csv]

with open('Split_Into_Types/Contains_img.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter="\t", quotechar='"')
    headers = next(csvreader)
    CWBR = namedtuple('CWBR', headers)
    for row in csvreader:
        item = CWBR(*row)
        cwbr_dict[item.ID] = item

In [3]:
broken_links = ['http://www.cwbr.com/civilwarbookreview/images/jackets/TheLeadershipOfUlyssesSGrant.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/BerryAllthatMakesaMan.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/Canaan.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/tintern.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/HLHunley.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/McPhersonDaysofDestiny.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/EdwardsShelbysExpedition.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/OutOfTheHouseOfBondage.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/RobertLee.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/WelfareAndCharity.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/ColonelsInBlue.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/TheOzarks.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/UpcountrySouthCarolinaGoesToWar.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/TheBattleOfPickettsMill.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/BurningRailsAsWePleased.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/SexSicknessandSlavery.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/QuantrillInTexas.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/OutOfTheMouthOfHell.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/FlagsCivilwarMissouri.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/HorsesandMules.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/LastToLeaveTheField.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/QueenOfTheConfederacy.jpg',
                "http://www.cwbr.com/civilwarbookreview/images/jackets/PrinceGeorge'sCountry.jpg",
                'http://www.cwbr.com/civilwarbookreview/images/jackets/CivilWarRevolvers.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/TheWilmington&WeldonRailroadInTheCivilWar.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/CivilWarSoldiersOfGreaterCleveland.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/LeesCavalrymen.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/GettingUsedToBeingShotAt.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/LifeandLabor.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/TheSeventhStarOfTheConfederacy.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/ARegimentOfSlaves.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/AcountryRiven.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/RebelsandRunaways.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/FloridasMonuments.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/KallenUndergroundRailroad.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/OpticTheBlueandTheGray.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/BrianThomsenAlternateGettysburg.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/VoicesFromCompanyD.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/GarlandTrailMakers.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/DurkinTheLastMan.jpg',
                'http://www.cwbr.com/civilwarbookreview/images/jackets/NeitherLadynorSlave.jpg',
                

                
               ]




import urllib.request
import os

path_regex = re.compile('src="(?P<path>[\S]+)"')

def find_img_url(image_text):
    imagepath = path_regex.search(image_text)
    webpath = imagepath.group('path')
    url = 'http://www.cwbr.com{}'.format(webpath)
    return url


for urn, pack in cwbr_dict.items():
    images = [pack.Image, pack.Image2, pack.Image3, pack.Image4, pack.Image5]
    print(images)
    image_urls = [find_img_url(i) for i in images if i]
    for image_url in image_urls:
        print(image_url)
        os.makedirs('images/{}'.format(urn), exist_ok=True)
        filepath = 'images/{}/{}'.format(urn, image_url.split('/')[-1])
        print(filepath)        
        if os.path.isfile(filepath):
            continue
        if image_url in broken_links:
            continue
        with urllib.request.urlopen(image_url) as response:
            binary = response.read()
        with open(filepath, 'bw') as f:
            f.write(binary)

['<img hspace=6 align=LEFT width=161 height=194 alt="Product Cover" border=0 src="/civilwarbookreview/images/jackets/ChristopherCollierSlaveryandthecomingoftheCivilWar.jpg">', '', '', '', '']
http://www.cwbr.com/civilwarbookreview/images/jackets/ChristopherCollierSlaveryandthecomingoftheCivilWar.jpg
images/1582/ChristopherCollierSlaveryandthecomingoftheCivilWar.jpg
['<img hspace=6 align=LEFT width=161 height=237 alt="Product Cover" border=0 src="/civilwarbookreview/images/jackets/TheFirstEmancipator.jpg">', '', '', '', '']
http://www.cwbr.com/civilwarbookreview/images/jackets/TheFirstEmancipator.jpg
images/3430/TheFirstEmancipator.jpg
['<img hspace=6 align=LEFT width=161 height=241 alt="Product Cover" border=0 src="/civilwarbookreview/images/jackets/DodyMyersEchoesoftheFallingSpring.jpg">', '', '', '', '']
http://www.cwbr.com/civilwarbookreview/images/jackets/DodyMyersEchoesoftheFallingSpring.jpg
images/2463/DodyMyersEchoesoftheFallingSpring.jpg
['<img hspace=6 align=LEFT width=161 hei

In [25]:
len(broken_links)

41

In [4]:
# prints the item with the longest Review

longest_item = ('', '')
for k, v in cwbr_dict.items():
    if len(v.Review) > len(longest_item[1]):
        longest_item = (k, v.Review)
print(k)

3206


In [5]:
def strip_and_replace(text):
    return text.strip().replace(":", "").replace(".", "").replace(",", "").replace('"', '').replace("<p>", "").lower().replace(' ', '')

inside, not_inside = 0, 0
for uid, issue in cwbr_dict.items():
    if issue.Teaser in issue.Title:
        inside += 1
    elif strip_and_replace(issue.Teaser[:-5]) in strip_and_replace(issue.Title):
        inside += 1
    else:
        not_inside += 1
        
#       is Teaser a subset of Title??
#         print("Teaser: {}\nTitle: {}\n\n".format(issue.Teaser, issue.Title))
# print(inside, not_inside)

In [6]:
# how many have Sub_headlines?  How do they Sub_headlines compare to Titles, Headlines?

count = 0
for uri, issue in cwbr_dict.items():
    if len(issue.Sub_headline):
        count += 1
#         print("{}\nS: {}\nH: {}\nT: {}\n\n".format(uri, issue.Sub_headline, issue.Headline, issue.Title))
# print(count)

In [7]:
# How many have a Headline?  Are Headlines a shortened form of the Title?

count = 0
similar = 0
for uri, issue in cwbr_dict.items():
    if len(issue.Headline):
        count += 1
        if strip_and_replace(issue.Headline) in strip_and_replace(issue.Title):
            similar += 1
#             print("{}\nS: {}\nH: {}\nT: {}\n\n".format(uri, issue.Sub_headline, issue.Headline, issue.Title))
# print(count, similar)

In [8]:
# what is our datastructure?  give an example

# item = cwbr_dict['6115']
# print(dir(item))
# for field in item._fields:
#     print("{}\t\t{}".format(field, getattr(item, field)[:100]))

In [9]:
# how many have a Teaser?  print all the Teasers to screen.

count = 0
for k, v in cwbr_dict.items():
    if len(v.Teaser):
        count += 1
#         print(v.Teaser)
# print(count)

In [10]:
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

In [11]:
# a styling with justified lines & indentation of some number of points.

styles=getSampleStyleSheet()
styles.add(ParagraphStyle(name='Justify',
                          alignment=TA_JUSTIFY,
                          firstLineIndent=25))


In [12]:
# script to turn the html into a pdf.  
# several different variations are included, to challenge our dataset -- to see which have errors.

# This one crashes on the first failure.

def do_thing(uri):
    issue = cwbr_dict[uri]
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)

    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')

    if len(split_strings) == 1:
        p = Paragraph(split_strings[0], styles["Justify"])
        template.append(p)
    else:
        for i in split_strings:
            p = Paragraph(i, styles["Justify"])
            template.append(p)
            template.append(Spacer(1,0.2*inch))

    doc.build(template)


In [13]:
# This one logs which ones fail, then continues to the next one.

def do_thing_log_fails(uri):
    issue = cwbr_dict[uri]
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)
    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')
    try:
        if len(split_strings) == 1:
            p = Paragraph(split_strings[0], styles["Justify"])
            template.append(p)
        else:
            for i in split_strings:
                if len(i):
                    p = Paragraph(i, styles["Justify"])
                    template.append(p)
                    template.append(Spacer(1,0.2*inch))
                else:
                    pass
        doc.build(template)
    except:
        failed.append(uri)

In [14]:
# another variety

def do_thing_without_exception(uri):
    issue = cwbr_dict[uri]
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)
    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')
    split_strings = [i.replace('</p>', '') for i in split_strings]
    for num, i in enumerate(split_strings):
        for tag in ('b', 'i', 'a'):  # in case of extra single-letter </tag> in string
            extra_tag = i.count('<{}>'.format(tag)) - i.count('</{}>'.format(tag))
            if extra_tag < 0:
                for _ in range(extra_tag):
                    print(split_strings)
                    last_pos = i.rfind('</{}>'.format(tag))
                    split_strings[num] = i[:last_pos] + i[last_pos+3:]
                    extra_closed_tag.append((uri, i))
            if extra_tag > 0:
                for _ in range(extra_tag):
                    split_strings[num] = i + '</{}>'.format(tag)
    if len(split_strings) == 1:
        p = Paragraph(split_strings[0], styles["Justify"])
        template.append(p)
    else:
        for i in split_strings:
            if len(i):
                p = Paragraph(i, styles["Justify"])
                template.append(p)
                template.append(Spacer(1,0.2*inch))
            else:
                pass
                print('Empty string: {}'.format(i))
    doc.build(template)


In [15]:
# This one prints to screen each step, to nail down when the data error is
# Only run this one as 'do_thing_print_fails('25322')
#   (E.g., for one item, else it will use all your RAM and freeze your computer)

def do_thing_print_fails(uri):
    issue = cwbr_dict[uri]
    print('a')
    doc = SimpleDocTemplate('output/{}.pdf'.format(uri),
                           pagesize=letter,
                           rightMargin=72,
                           leftMargin=72,
                           topMargin=72,
                           bottomMargin=18)
    print('b')
    template = []
    split_strings = issue.Review.replace('<br>', '<p>').replace('</br>', '</p>').split('<p>')
    print('c')
    try:
        if len(split_strings) == 1:
            print('d')
            p = Paragraph(split_strings[0], styles["Justify"])
            print('e')
            template.append(p)
            print('f')
        else:
            print('g')
            for i in split_strings:
                print('h')
                if len(i):
                    print('i')
                    print(i)
                    p = Paragraph(i, styles["Justify"])
                    print('j')
                    template.append(p)
                    print('k')
                    template.append(Spacer(1,0.2*inch))
                    print('l')
                else:
                    print('m')
                    pass
        print('n')
        doc.build(template)
        print('o')
    except:
        print('p')

In [16]:
# this runs all the items in the loaded csv, with logging fails into a list called "failed".

failed = []

for uri in cwbr_dict:
    do_thing_log_fails(uri)

In [17]:
# prints how many failed, how many are in the loaded csv, and what percent failed.

print(len(failed),
      len(cwbr_dict),
      '{}%'.format(len(failed)/len(cwbr_dict)//.01))


0 1636 0.0%


In [18]:
import sys

extra_closed_tag = []
parse_error = []

for uri in failed:
    try:
        do_thing_without_exception(uri)
    except Exception:
        parse_error.append((uri, sys.exc_info()))


print(len(failed))
if parse_error:
    print(parse_error[0])

# with open('exception.txt', 'w') as f:
#     output_text = ''
#     for item in parse_error:
#         output_text += '{}\n'.format(item)
#     f.write(output_text)



0


In [19]:
# this identifies 3 causes of errors in parsing the data -- and a Leftovers group for items with unknown error

Review_starts_with_url = []
Review_starts_with_image = []
Review_starts_with_header_url = []
Leftovers = []
Other_breaking_types = ['5738', '6050', '3326', '6176', '5804', '3453', ]


for item in parse_error:
#     if item[0] in Other_breaking_types:
#         continue
    if "<para><a href=" in item[1][1].args[0]:
        Review_starts_with_url.append(item)
    elif "<para><img " in item[1][1].args[0]:
        Review_starts_with_image.append(item)
    elif "<para><h3><a href=" in item[1][1].args[0]:
        Review_starts_with_header_url.append(item)
    else:
        Leftovers.append(item)

In [20]:
# prints the number of items in each category.  You want the numbers to end up "0" eventually.

print("failed:", len(failed), '\n',
      "parse_error", len(parse_error), '\n',
      "Review_starts_with_url", len(Review_starts_with_url), '\n',
      "Review_starts_with_header_url", len(Review_starts_with_header_url), '\n',
      "Review_starts_with_image:", len(Review_starts_with_image), "\n",
      "Remainder", len(parse_error)-len(Review_starts_with_url)-len(Review_starts_with_header_url), '\n',
      "Leftovers", len(Leftovers), '\n',) 

print(sorted(failed))

failed: 0 
 parse_error 0 
 Review_starts_with_url 0 
 Review_starts_with_header_url 0 
 Review_starts_with_image: 0 
 Remainder 0 
 Leftovers 0 

[]


In [21]:
# this command is useful now for finding the error in each 'nnnn' listed above.  The error is in the last printed line.

# do_thing_print_fails('4244')

In [22]:
# to see if all of an error type occur in one Record_type.
# replace Review_starts_with_image with whatever error you wish to examine.

url_starts = {"Annotation": 0, "Classics": 0, "Review": 0, "": 0, "Interview": 0}
for i in Review_starts_with_image:
    url_starts[cwbr_dict[i[0]].Record_type] += 1

if Review_starts_with_image:
    print(Review_starts_with_image[0])
    
print(url_starts)
    

{'': 0, 'Review': 0, 'Classics': 0, 'Annotation': 0, 'Interview': 0}


In [23]:
print(image_urls)

['http://www.cwbr.com/civilwarbookreview/images/jackets/WomenoftheCivilWar.jpg']


In [24]:
dir(cwbr_dict['4053'])


KeyError: '4053'